Part 1 - Import de todas as bibliotecas necessárias 

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Part 2 - Leitura do local dataframe + infos básicas

In [16]:
df = pd.read_csv('RTA Dataset.csv')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12316 entries, 0 to 12315
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Time                         12316 non-null  object
 1   Day_of_week                  12316 non-null  object
 2   Age_band_of_driver           12316 non-null  object
 3   Sex_of_driver                12316 non-null  object
 4   Educational_level            11575 non-null  object
 5   Vehicle_driver_relation      11737 non-null  object
 6   Driving_experience           11487 non-null  object
 7   Type_of_vehicle              11366 non-null  object
 8   Owner_of_vehicle             11834 non-null  object
 9   Service_year_of_vehicle      8388 non-null   object
 10  Defect_of_vehicle            7889 non-null   object
 11  Area_accident_occured        12077 non-null  object
 12  Lanes_or_Medians             11931 non-null  object
 13  Road_allignment              12

In [33]:
df.isnull().sum()   #verificar quantas colunas têm valores nulos

Time                              0
Day_of_week                       0
Age_band_of_driver                0
Sex_of_driver                     0
Educational_level               741
Vehicle_driver_relation         579
Driving_experience              829
Type_of_vehicle                 950
Owner_of_vehicle                482
Service_year_of_vehicle        3928
Defect_of_vehicle              4427
Area_accident_occured           239
Lanes_or_Medians                385
Road_allignment                 142
Types_of_Junction               887
Road_surface_type               172
Road_surface_conditions           0
Light_conditions                  0
Weather_conditions                0
Type_of_collision               155
Number_of_vehicles_involved       0
Number_of_casualties              0
Vehicle_movement                308
Casualty_class                    0
Sex_of_casualty                   0
Age_band_of_casualty              0
Casualty_severity                 0
Work_of_casuality           

In [18]:
df.dtypes

Time                           object
Day_of_week                    object
Age_band_of_driver             object
Sex_of_driver                  object
Educational_level              object
Vehicle_driver_relation        object
Driving_experience             object
Type_of_vehicle                object
Owner_of_vehicle               object
Service_year_of_vehicle        object
Defect_of_vehicle              object
Area_accident_occured          object
Lanes_or_Medians               object
Road_allignment                object
Types_of_Junction              object
Road_surface_type              object
Road_surface_conditions        object
Light_conditions               object
Weather_conditions             object
Type_of_collision              object
Number_of_vehicles_involved     int64
Number_of_casualties            int64
Vehicle_movement               object
Casualty_class                 object
Sex_of_casualty                object
Age_band_of_casualty           object
Casualty_sev

In [22]:
no_veh = df['Number_of_vehicles_involved']
print(no_veh)

no_cas = df['Number_of_casualties']
print(no_cas)

0        2
1        2
2        2
3        2
4        2
        ..
12311    2
12312    2
12313    1
12314    2
12315    2
Name: Number_of_vehicles_involved, Length: 12316, dtype: int64
0        2
1        2
2        2
3        2
4        2
        ..
12311    1
12312    1
12313    1
12314    1
12315    1
Name: Number_of_casualties, Length: 12316, dtype: int64


Part 3 - Analisar e eliminar colunas não importantes

In [88]:
#.unique() -> ver todos os valores de uma coluna

defect_of_vehicle = df['Defect_of_vehicle'].count() #eliminar -> muitas entradas a NULL e não acrescentam info
print(defect_of_vehicle)

type_of_vehicle = df['Type_of_vehicle'].unique()
print(type_of_vehicle, "\n\n")

lanes_or_medians = df['Lanes_or_Medians'].unique()
print(lanes_or_medians, "\n\n")

owner_of_vehicle = df['Owner_of_vehicle'].unique()  #parece me pouco relevante
print(owner_of_vehicle, "\n\n")

service_year_of_vehicle = df['Service_year_of_vehicle'].unique()
print(service_year_of_vehicle, "\n\n")

defect_of_vehicle = df['Defect_of_vehicle'].value_counts()  #tem 7777 "no defect"
print(defect_of_vehicle, "\n\n")    #eliminar -> parece me muito inútil

area_accident_occured = df['Area_accident_occured'].value_counts()
print(area_accident_occured, "\n\n")

road_allignment = df['Road_allignment'].value_counts()  #parece me pouco relevante!
print(road_allignment, "\n\n")

types_of_junction = df['Types_of_Junction'].value_counts()  #parece me muito pouco relevante
print(types_of_junction, "\n\n")

vehicle_movement = df['Vehicle_movement'].value_counts()    #será útil?
print(vehicle_movement, "\n\n")

pedestrian_movement = df['Pedestrian_movement'].value_counts()  #será importante?
print(pedestrian_movement, "\n\n")

number_of_vehicles_involved = df['Number_of_vehicles_involved'].value_counts()  #será importante?
print(number_of_vehicles_involved, "\n\n")

number_of_casualties = df['Number_of_casualties'].value_counts()    #será importante?
print(number_of_casualties, "\n\n")

casualty_class = df['Casualty_class'].value_counts()    #eliminar -> parece que não acrescenta info    
print(casualty_class, "\n\n")

sex_of_casualty = df['Sex_of_casualty'].value_counts()  #pouco importante e tem imensos valores NA
print(sex_of_casualty, "\n\n")

age_band_of_casualty = df['Age_band_of_casualty'].value_counts() #pouco importante e tem imensos valores NA
print(age_band_of_casualty, "\n\n")

casualty_severity = df['Casualty_severity'].value_counts()  #o que é que isto significa sequer?
print(casualty_severity, "\n\n")

work_of_causalty = df['Work_of_casuality'].value_counts()  #info confusa
print(work_of_causalty, "\n\n")

fitness_of_casuality = df['Fitness_of_casuality'].value_counts()  #coisa mais inútil daqui
print(fitness_of_casuality, "\n\n")

['Automobile' 'Public (> 45 seats)' 'Lorry (41?100Q)' nan
 'Public (13?45 seats)' 'Lorry (11?40Q)' 'Long lorry' 'Public (12 seats)'
 'Taxi' 'Pick up upto 10Q' 'Stationwagen' 'Ridden horse' 'Other' 'Bajaj'
 'Turbo' 'Motorcycle' 'Special vehicle' 'Bicycle'] 


[nan 'Undivided Two way' 'other' 'Double carriageway (median)' 'One way'
 'Two-way (divided with solid lines road marking)'
 'Two-way (divided with broken lines road marking)' 'Unknown'] 


['Owner' 'Governmental' nan 'Organization' 'Other'] 


['Above 10yr' '5-10yrs' nan '1-2yr' '2-5yrs' 'Unknown' 'Below 1yr'] 


Defect_of_vehicle
No defect    7777
7              76
5              36
Name: count, dtype: int64 


Area_accident_occured
Other                              3819
Office areas                       3451
Residential areas                  2060
 Church areas                      1060
 Industrial areas                   456
School areas                        415
  Recreational areas                327
 Outside rural areas  

In [59]:
#value.counts diz o número de vezes que cada y aparece para cada x

teste = df.loc[:, ['Owner_of_vehicle', 'Service_year_of_vehicle']].value_counts()
print(teste)

Owner_of_vehicle  Service_year_of_vehicle
Owner             Unknown                    2606
                  2-5yrs                     1571
                  Above 10yr                 1179
                  5-10yrs                    1145
                  1-2yr                       655
Governmental      Unknown                     201
Owner             Below 1yr                   182
Governmental      2-5yrs                      115
                  Below 1yr                   100
                  1-2yr                        93
Organization      2-5yrs                       84
Governmental      Above 10yr                   83
                  5-10yrs                      73
Organization      Unknown                      66
                  5-10yrs                      62
                  Above 10yr                   51
                  1-2yr                        37
Other             1-2yr                        12
                  Unknown                      10
Name: co

Part 4 - Renomear nomes de colunas para melhorar elegibilidade

In [ ]:
#time, day_of_week, age_band_of_driver, sex_of_driver, educational_level, vehicle_driver_relation, driving_experience
#type_of_vehicle, owner_of_vehicle, service_year_of_vehicle, defect_of_vehicle, area_accident_occured
#lanes_or_medians, road_allignment, types_of_junction, road_surface_type, road_surface_conditions, ligh_conditions
#weather_conditions, type_of_collision, number_of_vehicles_involved, number_of_casualties, vehicle_movement
#casualty_class, sex_of_casualty, age_band_of_casualty, casualty_severity, work_of_casualty, fitness_of_casualty
#pedestrian_movement, cause_of_accident, accident_severity, 
colunas_renomeadas = []

#df.columns = colunas_renomeadas
#df

In [96]:
#X
X = df.iloc[:, :-1].values # todas as colunas menos a target
#Y
Y =  df.iloc[:, -1].values # coluna target

print("Número de exemplos nos nossos dados: ", X.shape[0])

from sklearn.ensemble import ExtraTreesClassifier

#model = ExtraTreesClassifier()
#model.fit(X,Y)

#feat_importances = pd.Series(model.feature_importances_, index=df.columns[:-1])
#feat_importances.nlargest(8).plot(kind='barh')

#plt.show()


['Slight Injury' 'Slight Injury' 'Serious Injury' ... 'Serious Injury'
 'Slight Injury' 'Slight Injury']
Número de exemplos nos nossos dados:  12316
